## 🛠️ Mod5 Data Challenge 1: ETL-in-the-Wild — Cleaning Civic Data for Product Insights

### 🎯 Purpose

Practice applying the "Transform" step of an ETL pipeline using a real-world dataset—NYC 311 Service Requests. You’ll clean, standardize, and engineer features to prep the data for downstream analysis and business reporting.

### 📚 KSBs

K14 – Strategic feature engineering to improve analysis outcomes

S5 – Data transformation techniques in Python

S8 – Uses Pandas for EDA and data manipulation

B4 – Exercises critical judgment about data quality and reliability

B6 – Pursues deeper insights beyond surface-level observations

### Data
Use the **nyc311.csv** file located in your Github's `data` folder within Mod5/DataChallenges.  This is a sample of the originial file looking at just one week of data since the dataset is HUGE.  Read more about the columns [HERE](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9/about_data).   

### 👩‍🏫 Instructor-Led Demo (15 minutes)

#### Step 1:  Load + Preview 
* Read in data

* Show .head(), .info() to examine structure and datatypes

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('/Users/Marcy_Student/DA2025_Lectures/DA2025_Lectures/Mod5/DataChallenges/data/nyc311.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55815 entries, 0 to 55814
Data columns (total 41 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unique Key                      55815 non-null  int64  
 1   Created Date                    55815 non-null  object 
 2   Closed Date                     41348 non-null  object 
 3   Agency                          55815 non-null  object 
 4   Agency Name                     55815 non-null  object 
 5   Complaint Type                  55815 non-null  object 
 6   Descriptor                      54400 non-null  object 
 7   Location Type                   49701 non-null  object 
 8   Incident Zip                    55361 non-null  float64
 9   Incident Address                54235 non-null  object 
 10  Street Name                     54233 non-null  object 
 11  Cross Street 1                  46608 non-null  object 
 12  Cross Street 2                  

#### Step 2:  Clean up Categorical Variable 

* Normalize Borough & Complaint Type (e.g., strip whitespace, title-case or upper-case values)

* Show .value_counts() before and after cleaning

In [36]:
# Strip whitespace and standardize case
for col in ["Complaint Type", "Borough"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

# pick a style (title-case often reads nicely)
df["Complaint Type"] = df['Complaint Type'].str.title()
df["Borough"] = df['Borough'].str.title()

# quick sanity checks
display(df["Complaint Type"].value_counts().head(10))
display(df["Borough"].value_counts())

Complaint Type
None    41348
Name: count, dtype: int64

Borough
None    41348
Name: count, dtype: int64

#### Step 3:  Feature Engineering

* Convert Created Date to datetime

* Create a new feature: is_weekend based on weekday

In [34]:
df["Created Date"] =pd.to_datetime(df['Created Date'])

# is_weekend flag

df["is_weekend"] = df['Created Date'].dt.day_of_week > 5 

### 👩‍💻 Student-Led Section (30 minutes)

#### Task1:  Clean & Filter

* Drop rows with missing Created Date or Closed Date

* Drop duplicate rows

* Filter data to only include one borough (e.g., Brooklyn)

In [50]:
# 1a) Drop rows with missing Created/Closed Date
df = df.dropna (subset=['Created Date','Closed Date'])

# 1b) Drop duplicates
df = df.drop_duplicates()

# 1c) Filter to a single borough (STUDENT CHOICE)
borough_name = 'Brooklyn' # e.g., "Brooklyn"
df_b = df[df["Borough"] == borough_name].copy()
df_b.shape

(0, 42)

In [18]:
df.isnull().sum()

Unique Key                            0
Created Date                          0
Closed Date                           0
Agency                                0
Agency Name                           0
Complaint Type                        0
Descriptor                         1264
Location Type                      3507
Incident Zip                        316
Incident Address                   1017
Street Name                        1018
Cross Street 1                     3474
Cross Street 2                     3438
Intersection Street 1              4481
Intersection Street 2              4441
Address Type                        142
City                               2423
Landmark                           7430
Facility Type                     41218
Status                                0
Due Date                          41340
Resolution Description              646
Resolution Action Updated Date        0
Community Board                       0
BBL                                5016


#### Task2:  Time Transformations 

* Create a new column `response_time_hrs` = difference between Closed Date and Created Date (in hours)

* Ensure datetime types are properly parsed and timezone-aware (localize to America/New_York → convert to UTC)

In [19]:
# RUN THIS CELL WITHOUT CHANGES!!!! 

LOCAL_TZ = "America/New_York"

def to_utc(series, local_tz=LOCAL_TZ):
    # 1) parse
    s = pd.to_datetime(series, errors="coerce")

    # 2) if naive -> localize; if tz-aware -> skip (KEEP NONE in this if statement)
    if s.dt.tz is None:
        s = s.dt.tz_localize(local_tz, nonexistent="shift_forward", ambiguous="NaT")

    # 3) convert whatever tz it has to UTC
    return s.dt.tz_convert("UTC")

In [39]:
# Convert both columns to utc
df_b["Created Date"] = to_utc(df_b["Created Date"])
df_b["Closed Date"]  = to_utc(df_b['Closed Date'])

# Now compute the response time
delta = df_b["Closed Date"] - df_b["Created Date"]
df_b["response_time_hrs"] = delta.dt.total_seconds() / 3600

# Check the data with a .head()


#### Task3:  Feature Engineer

* Create `hour_of_day` from Created Date

* Create `is_high_priority`: True if Complaint Type is in a list of priority complaints (e.g., ["HEAT/HOT WATER", "ELECTRIC", "STRUCTURAL"])

In [54]:
# hour_of_day from Created Date 
df_b["hour_of_day"] = df['Created Date'].dt.hour

# define a high-priority list (STUDENT CHOICE)
priority_list = ['Electric','Structural' ] # e.g., ["Heat/Hot Water","Electric","Elevator","Structural"
df_b["is_high_priority"] = df_b['Complaint Type'].isin(priority_list)

display(df_b[["Complaint Type","hour_of_day","is_high_priority"]].head(5))
print ( df['Complaint Type'])

,Complaint Type,hour_of_day,is_high_priority
7,NaN,1,False
37,NaN,1,False
43,NaN,1,False
47,NaN,1,False
50,NaN,1,False


7        None
37       None
43       None
47       None
50       None
         ... 
55810    None
55811    None
55812    None
55813    None
55814    None
Name: Complaint Type, Length: 41348, dtype: object


#### Task4:  Aggregates & Visuals 

* Average response time by Complaint Type (sorted descending)

* Total number of complaints by hour_of_day (bar chart)

In [ ]:
# Average response time by complaint type (descending)
avg_resp = (df_b)

In [ ]:
# Total complaints by hour_of_day (bar chart)
None

#### Notebook Reflection

Answer the questions in a markdown cell below.  Be thorough (no need to use AWES just answer the question fully)

1. Describe one transformation or feature you engineered (e.g., response_time_hrs, is_high_priority) and explain why it would be useful for a city analyst or operations manager.

2. What could go wrong if you didn’t clean the datetime fields properly or skipped removing rows with missing Closed Date? How might that distort future dashboards or decisions?

### 📣 Class Share-Out (15 minutes)

Be ready to share out the following points with the class: 

📣 Explain:

How average response_time_hrs by complaint type offers insight (e.g., “Elevator complaints take longer to resolve”)

What does your chosen borough trends reveal (e.g., “Most heating complaints come in before 10am”)

🛠️ Propose:

One next transformation step you would include in the full data pipeline (e.g., flagging overdue requests)

